# Sentiment Analysis: Donald Trump and Joe Biden Reelection Run in 2024

## Description

This part of the project will be the develop of the code to gather the real time data from the Twitter API. The data will be used to perform a sentiment analysis of the tweets about Donald Trump and Joe Biden reelection run in 2024.



## Gathering Data

In [1]:
import json
with open("C:/Users/diazi/.secret/twitter_api.json") as f:
    keys = json.load(f)

keys.keys()

dict_keys(['consumer-key', 'consumer-key-secret', 'access-token', 'access-token-secret', 'bearer-token'])

In [2]:
## tokens
bearer_token = keys['bearer-token']
consumer_key = keys['consumer-key']
consumer_secret = keys['consumer-key-secret']
access_key = keys['access-token']
access_secret = keys['access-token-secret']

**Setting up the connection to the Twitter API**

In [3]:
import csv
import json
import time
from datetime import datetime
import requests

In [5]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")

In [6]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

In [7]:

def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

In [8]:
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))

In [9]:
def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "(Donald Trump) -is:retweet -is:reply lang:en place_country:GB", "tag": "politics"}
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

In [10]:

def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream?tweet.fields=attachments,author_id,created_at,public_metrics,source&expansions=attachments.media_keys&media.fields=public_metrics,duration_ms", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent=4, sort_keys=True))

            #saving into json file
            with open(file_name, 'a') as f:
                json.dump(json_response, f)
                f.write('\n')



In [11]:
def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)

In [ ]:
if __name__ == "__main__":
    main()

{"data": [{"id": "1652869083526840322", "value": "(Donald Trump) -is:retweet -is:reply lang:en place_country:GB", "tag": "politics"}], "meta": {"sent": "2023-05-01T03:08:29.411Z", "result_count": 1}}
{"meta": {"sent": "2023-05-01T03:08:30.395Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "(Donald Trump) -is:retweet -is:reply lang:en place_country:GB", "tag": "politics", "id": "1652872596537393153"}], "meta": {"sent": "2023-05-01T03:08:32.272Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200
{
    "data": {
        "attachments": {},
        "author_id": "4714672827",
        "created_at": "2023-05-01T07:45:35.000Z",
        "edit_history_tweet_ids": [
            "1652942328418430976"
        ],
        "id": "1652942328418430976",
        "public_metrics": {
            "impression_count": 0,
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 0
        },
        "tex